In [ ]:
!pip install transformers datasets evaluate;

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.8 MB 30.6 MB/s 
     |████████████████████████████████| 451 kB 65.9 MB/s 
     |████████████████████████████████| 72 kB 643 kB/s 
     |████████████████████████████████| 182 kB 37.7 MB/s 
     |████████████████████████████████| 7.6 MB 55.0 MB/s 
     |████████████████████████████████| 212 kB 28.8 MB/s 
     |████████████████████████████████| 132 kB 60.4 MB/s 
     |████████████████████████████████| 127 kB 30.1 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import json
import os
import logging
import pandas as pd

import torch
import torch.nn as nn
import evaluate

from datasets import load_dataset, Dataset, DatasetDict
from transformers import AutoTokenizer, DataCollatorWithPadding, AutoModelForSequenceClassification, TrainingArguments, Trainer

os.environ["TOKENIZERS_PARALLELISM"] = "true"

In [ ]:
def get_dataset(path_to_data):
  with open(path_to_data) as f:
    data = json.loads(f.read())
  data = [[' </s> '.join(x['tweets']), 1 if x['label']=='bot' else 0] for x in data]
  df = pd.DataFrame(data)
  df.columns = ["text", "label"]
  data = Dataset.from_pandas(df)
  return data

In [ ]:
data_args = {'path_to_train' : '/content/drive/MyDrive/advNLP/twibot20/train.json',
             'path_to_test' : '/content/drive/MyDrive/advNLP/twibot20/test.json',
             'path_to_val' : '/content/drive/MyDrive/advNLP/twibot20/val.json'}

max_length = 128
checkpoint = "vinai/bertweet-base"

In [ ]:
train_data = get_dataset(data_args['path_to_train'])
val_data = get_dataset(data_args['path_to_val'])
test_data = get_dataset(data_args['path_to_test'])

data = DatasetDict({
    'train': train_data,
    'test': test_data,
    'valid': val_data})

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

Could not locate the tokenizer configuration file, will try to use the model config instead.
loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--roberta-base/snapshots/ff46155979338ff8063cdad90908b498ab91b181/config.json
Model config RobertaConfig {
  "_name_or_path": "roberta-base",
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "4.25.1",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 50265
}

loading file vocab.json from cache at /root/.cache/huggingface/hub/mo

In [ ]:
def preprocess_function(examples):
  return tokenizer(examples["text"], truncation=True, padding=True, max_length=max_length)

tokenized_dataset = data.map(preprocess_function, batched=True)

  0%|          | 0/9 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

  0%|          | 0/3 [00:00<?, ?ba/s]

In [ ]:
import numpy as np

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
accuracy = evaluate.load("accuracy")

def compute_metrics(eval_pred):
  predictions, labels = eval_pred
  predictions = np.argmax(predictions, axis=1)
  return accuracy.compute(predictions=predictions, references=labels)

In [ ]:
id2label = {0: 0, 1: 1}
label2id = {0: 0, 1: 1}

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, 
                                                           num_labels=2, 
                                                           id2label=id2label, 
                                                           label2id=label2id)


freeze_pretrained_layers = True

if freeze_pretrained_layers:
  for param in model.base_model.parameters():
      param.requires_grad = False

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--roberta-base/snapshots/ff46155979338ff8063cdad90908b498ab91b181/config.json
Model config RobertaConfig {
  "_name_or_path": "roberta-base",
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": 0,
    "1": 1
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "0": 0,
    "1": 1
  },
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "4.25.1",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 50265
}

loading weights file pytorch_model.bin from cache at /root/.cache/huggingf

In [ ]:
training_args = TrainingArguments(
    output_dir="/content/drive/MyDrive/advNLP/outputs/tweets/bertweet",
    learning_rate=4e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=10,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=False,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["valid"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [ ]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: text. If text are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 8278
  Num Epochs = 10
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 10350
  Number of trainable parameters = 592130
You're using a RobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a

Epoch,Training Loss,Validation Loss,Accuracy
1,0.672100,0.663927,0.611416
2,0.656400,0.646379,0.635518
3,0.644800,0.634326,0.629175
4,0.633700,0.620039,0.668076
5,0.626600,0.612360,0.678647
6,0.624800,0.608490,0.682452


The following columns in the evaluation set don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: text. If text are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 2365
  Batch size = 8
Saving model checkpoint to /content/drive/MyDrive/advNLP/outputs/tweets/roberta/checkpoint-1035
Configuration saved in /content/drive/MyDrive/advNLP/outputs/tweets/roberta/checkpoint-1035/config.json
Model weights saved in /content/drive/MyDrive/advNLP/outputs/tweets/roberta/checkpoint-1035/pytorch_model.bin
tokenizer config file saved in /content/drive/MyDrive/advNLP/outputs/tweets/roberta/checkpoint-1035/tokenizer_config.json
Special tokens file saved in /content/drive/MyDrive/advNLP/outputs/tweets/roberta/checkpoint-1035/special_tokens_map.json
The following columns in the evaluation set don't have a corresponding argument in `RobertaForSequenceClassifica

In [ ]:
from evaluate import evaluator

task_evaluator = evaluator("text-classification")

test_results = task_evaluator.compute(
    model_or_pipeline=model,
    tokenizer=tokenizer,
    data=tokenized_dataset['test'],
    metric=evaluate.combine(["accuracy", "recall", "precision", "f1"]),
    label_mapping={0: 0, 1: 1}
)

Could not locate the tokenizer configuration file, will try to use the model config instead.
loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--roberta-base/snapshots/ff46155979338ff8063cdad90908b498ab91b181/config.json
Model config RobertaConfig {
  "_name_or_path": "roberta-base",
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "4.25.1",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 50265
}

loading file vocab.json from cache at /root/.cache/huggingface/hub/mo

In [ ]:
test_results

{'accuracy': 0.6542688081149619,
 'recall': 0.859375,
 'precision': 0.6329113924050633,
 'f1': 0.728959575878065,
 'total_time_in_seconds': 62.13790452600006,
 'samples_per_second': 19.038298909886848,
 'latency_in_seconds': 0.05252570120541002}

In [ ]:
eval_results = task_evaluator.compute(
    model_or_pipeline=model,
    tokenizer=tokenizer,
    data=tokenized_dataset['valid'],
    metric=evaluate.combine(["accuracy", "recall", "precision", "f1"]),
    label_mapping={0: 0, 1: 1}
)

Could not locate the tokenizer configuration file, will try to use the model config instead.
loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--roberta-base/snapshots/ff46155979338ff8063cdad90908b498ab91b181/config.json
Model config RobertaConfig {
  "_name_or_path": "roberta-base",
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "4.25.1",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 50265
}

loading file vocab.json from cache at /root/.cache/huggingface/hub/mo

In [ ]:
eval_results

{'accuracy': 0.678646934460888,
 'recall': 0.8894858019953953,
 'precision': 0.6529577464788733,
 'f1': 0.7530864197530863,
 'total_time_in_seconds': 109.66953663699996,
 'samples_per_second': 21.564785194889787,
 'latency_in_seconds': 0.04637189709809724}